## Terms of Trade

In [1]:
import pandas as pd
import numpy as np

In [2]:
def basic_cleaning_stuff(path_trade):
    
    df_trade = pd.read_excel(path_trade, header=7)
    df_trade = df_trade.iloc[0:4]
    df_trade.drop(columns=['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 3', 'Scale'], inplace=True)
    df_trade = df_trade.transpose().reset_index()
    df_trade.columns = df_trade.iloc[0]
    df_trade = df_trade.drop([0])
    df_trade = df_trade.replace('...', np.nan)
    df_trade.columns = ['Indicator', 'Exports', 'Exports (national)', 'Imports', 'Imports (national)']

    return df_trade

In [28]:
def if_nans(path_trade):

    df_trade = pd.read_excel(path_trade, header=7)
    df_trade = df_trade.iloc[0:6]
    df_trade.drop(columns=['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 3', 'Scale'], inplace=True)
    df_trade = df_trade.transpose().reset_index()
    df_trade.columns = df_trade.iloc[0]
    df_trade = df_trade.drop([0])
    df_trade = df_trade.drop(columns=['Goods, Value of Imports, CIF,  US Dollars', 
                                        'Goods, Value of Imports, CIF, National Currency'])
    df_trade = df_trade.replace('...', np.nan)
    df_trade.columns = ['Indicator', 'Exports', 'Exports (national)', 'Imports (national)', 'Imports']

    return df_trade

In [18]:
def cleaning_if_nans(df, number):

    df2 = df.iloc[:number]

    df_fill = pd.DataFrame(df2['Imports']/3).fillna(method='ffill')

    df2['Imports'] = df_fill['Imports']

    df3 = df.iloc[number:]

    df_nonans = df2.append(df3)
    df_nonans

    #to drop the quaterly rows 
    del_num = [1]

    for i in list(range(1, int(df_nonans.shape[0]/4))):
        del_num.append(1+(4*i))

    df_nonans = df_nonans.drop(del_num)

    df_nonans.reset_index(inplace=True, drop=True)

    return df_nonans

In [19]:
def trade_terms_remove_Ms(df, country_code):
    
    df[f'{country_code}_trade'] = df['Exports']/df['Imports']

    df['index_t'] = df['Indicator'].apply(lambda x: x.replace('M', '-'))
    df['index_t'] = pd.to_datetime(df['index_t'])

    df['year'] = df['index_t'].dt.year
    df['month'] = df['index_t'].dt.month

    df = df.drop(columns=['Indicator', 'Exports', 'Exports (national)', 'Imports', 'Imports (national)', 'index_t'])

    return df

In [20]:
def append_df(df, df_trade):
    
    df = pd.merge(df, df_trade, left_on=['month', 'year'], right_on=['month', 'year'])

    return df

In [6]:
def data_combine(path, df_trade):
    
    exchange_df = pd.read_csv(path) #still 4997 rows
    print(exchange_df.shape)

    df_with_trade = pd.merge(exchange_df, df_trade, left_on=['month', 'year'], right_on=['month', 'year'])

    return df_with_trade

In [155]:
path_trade = '<path to trade file>'
country_code = 'MXN'

In [156]:
df_clean = basic_cleaning_stuff(path_trade)
print(df_clean.isna().sum())
df_clean #240 rows

Indicator              0
Exports                0
Exports (national)     0
Imports               11
Imports (national)    11
dtype: int64


,Indicator,Exports,Exports (national),Imports,Imports (national)
1,2000M01,11263.489920,106770,1309.802489,12416.010738
2,2000M02,13157.448971,124280,1489.658112,14070.714666
3,2000M03,13564.044364,126090,1567.953316,14575.537233
4,2000M04,12342.663310,115710,1420.360124,13315.592091
5,2000M05,14637.455563,139170,1673.035920,15906.890916
...,...,...,...,...,...
236,2019M08,40185.608690,787220,NaN,NaN
237,2019M09,37032.847199,726740,NaN,NaN
238,2019M10,40617.756233,786770,NaN,NaN
239,2019M11,37460.542888,722730,NaN,NaN


In [157]:
#only if there are NaN values in either Exports or Imports, national values do not matter
df_clean = if_nans(path_trade)
print(df_clean.isna().sum())
df_clean #240 rows

Indicator             0
Exports               0
Exports (national)    0
Imports (national)    0
Imports               0
dtype: int64


,Indicator,Exports,Exports (national),Imports (national),Imports
1,2000M01,11263.489920,106770,112520,11870.074795
2,2000M02,13157.448971,124280,126630,13406.242060
3,2000M03,13564.044364,126090,129300,13909.357889
4,2000M04,12342.663310,115710,119370,12733.071639
5,2000M05,14637.455563,139170,144280,15174.909022
...,...,...,...,...,...
236,2019M08,40185.608690,787220,779320,39782.333483
237,2019M09,37032.847199,726740,729710,37184.190948
238,2019M10,40617.756233,786770,800040,41302.832716
239,2019M11,37460.542888,722730,707930,36693.429257


In [158]:
df_trade = trade_terms_remove_Ms(df_clean, country_code)
df_trade #240 rows

,MXN_trade,year,month
1,0.948898,2000,1
2,0.981442,2000,2
3,0.975174,2000,3
4,0.969339,2000,4
5,0.964583,2000,5
...,...,...,...
236,1.010137,2019,8
237,0.995930,2019,9
238,0.983413,2019,10
239,1.020906,2019,11


In [144]:
#use only for the first instance, comment out afterwards
df = df_trade.copy()

In [159]:
#start using from the second instance
df = append_df(df, df_trade)

In [164]:
df #240 rows

,AUD_trade,year,month,NZD_trade,GBP_trade,BRL_trade,CND_trade,CNY_trade,IDR_trade,KRW_trade,ZAR_trade,DKK_trade,JPY_trade,NOK_trade,SEK_trade,CHF_trade,USD_trade,MXN_trade
0,0.842293,2000,1,1.003388,0.813694,0.508670,0.945638,1.099882,2.025361,0.966169,1.046692,1.038130,1.173962,1.910093,1.154959,0.990176,0.639998,0.948898
1,0.951462,2000,2,1.019308,0.837567,0.544813,0.961176,1.100760,2.260810,1.054225,1.026758,1.137418,1.398738,1.576047,1.220729,1.020301,0.646103,0.981442
2,0.913350,2000,3,1.016901,0.888848,0.547213,0.926761,1.130967,2.185928,1.015339,1.015863,1.072170,1.307467,1.470429,1.244027,0.878031,0.646959,0.975174
3,1.033358,2000,4,0.992434,0.809538,0.561966,0.947427,1.117570,2.142900,1.013263,1.099865,1.084597,1.352430,1.547469,1.223735,0.968030,0.643754,0.969339
4,0.889007,2000,5,1.076697,0.850124,0.559707,0.941294,1.185489,2.038107,1.101935,1.018162,1.135144,1.175503,1.689195,1.207358,1.017391,0.619251,0.964583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1.308287,2019,8,0.712197,0.696799,0.301399,0.972375,1.193524,1.006536,1.036809,1.039426,1.147443,0.975805,1.094351,0.953020,1.166200,0.649857,1.010137
236,1.266188,2019,9,0.768793,0.712402,0.288366,0.974369,1.222150,0.987146,1.151971,1.042874,1.146786,0.980137,0.981220,1.008655,1.147144,0.655690,0.995930
237,1.096374,2019,10,0.828207,0.706762,0.270958,0.969341,1.253199,1.008292,1.126876,1.022916,1.124232,1.001700,1.144655,0.972321,1.187989,0.672099,0.983413
238,1.225906,2019,11,0.868980,0.715873,0.289389,0.987925,1.206784,0.909000,1.081305,1.050991,1.190842,0.986334,1.365512,1.020171,1.230501,0.677949,1.020906


In [161]:
df.to_csv('<path to save trade dataset>', index=False)

- **new_path**: path of new main dataset 2, created from the 'load_data_gdp' notebook

In [162]:
new_path = '<path of new main dataset 2>'

In [165]:
df_with_trade = data_combine(new_path, df)
print(df_with_trade.shape) #4997 rows
print(df_with_trade.isna().sum())


(4997, 53)
(4997, 69)
Time Series    0
AUD_USD        0
NZD_USD        0
GBP_USD        0
BRL_USD        0
              ..
NOK_trade      0
SEK_trade      0
CHF_trade      0
USD_trade      0
MXN_trade      0
Length: 69, dtype: int64


In [167]:
df_with_trade.to_csv('<path to save the new main dataset 3>', index=False)